In [1]:
import subprocess
import cv2
from subprocess import check_output
import numpy as np
from scipy import spatial

from transform.calibration import *

In [2]:
## parameters
proj_h = 720
proj_w = 1280

## init settings
circles = []
refPt = []
warp = False

Code structure-

Read Book:
    Convert PDF to Images
    Show page 1 (with transformation)
    while (!stopReading) :
        pageChange = Read Output Sensor
        if pageChange == nextPage :
            Read Page(+1)
        else if pageChange == prevPage :
            Read Page(-1)

In [3]:
def get_num_pages(pdf_path):
    output = check_output(["pdfinfo", pdf_path]).decode()
    pages_line = [line for line in output.splitlines() if "Pages:" in line][0]
    num_pages = int(pages_line.split(":")[1])
    return num_pages

In [4]:
def ConvertPdfToImages (pdfFile, imageFile) :
    
    # Path to Poppler's libraries (Download from https://blog.alivate.com.au/poppler-windows/)
    pdfToPPMPath = r"C:\Program Files (x86)\Poppler\poppler-0.68.0\bin\pdftoppm.exe"

    # Convert PDF to images
    subprocess.Popen('"%s" -png %s %s' % (pdfToPPMPath, pdfFile, imageFile))
    
    # Return number of pages in PDF
    return get_num_pages(pdfFile)

In [5]:
def ReadImg(imageFile):
    img = cv2.imread(imageFile)
    height, width = img.shape[:2]
    # resize img
    ratio = max(width/proj_w, height/proj_h);
    img = cv2.resize(img, (int(width/ratio), int(height/ratio)))
    return img

def ReadPage (imageFile,pageNumber, mat) :
    filename = imageFile + '-' + str(pageNumber) + '.jpg'
    img = ReadImg(filename)
    # Open CV code for transformation
    img = cv2.warpPerspective(img, mat, (proj_w, proj_h))
    cv2.imshow(fr_trans, img)
    
def initFrame(frameName):
    cv2.namedWindow(fr_trans)


In [6]:
def ReadSensorOutput () :
    return 0
    # Return 1 for next page, -1 for prev page based on threshold

In [7]:
def ReadBook (pdfFile) :
    global stop
    imageFile = pdfFile[:-4] # Prefix of image is PDF file name
    totalPages = ConvertPdfToImages(pdfFile, imageFile)
    pageNumber = 1
    ReadPage(imageFile,pageNumber)
    while (stopReading != 0) : # Replace with a listener to signal end of reading
        pageChange = ReadSensorOutput()
        if pageChange==1 and pageNumber<totalPages : 
            ReadPage(imageFile,pageNumber+1)
        elif pageChange==-1 and pageNumber>0 :
            ReadPage(imageFile,pageNumber-1)

In [8]:
if __name__ == "__main__":
    pdfFile = "258report.pdf"
    # read calibration image
    filename = "img2.jpg"
    img = ReadImg(filename)
    # calibration
    mat = calibration(img)
    # start to print
    fr_trans = "Perspective transformation"
    initFrame(fr_trans)
    while True:
        ReadPage("test",0, mat)
        key = cv2.waitKey(1)
        if key == 27:
            print("stop")
            break
 
    cv2.destroyAllWindows()
    
    
# ReadBook(pdfFile)


warp
pos:  [732 103] . nearest!  1
warp
pos:  [38 47] . nearest!  0
warp
pos:  [145 576] . nearest!  2
stop
stop
